# Importing relevant packages

In [170]:
import urllib.request
import types
import pandas as pd
import numpy as np
import ibm_boto3
import requests # library to handle requests
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import geopy
from botocore.client import Config
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from bs4 import BeautifulSoup 
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


# import k-means from clustering stage
from sklearn.cluster import KMeans

## Extracting neighborhoods with postal codes of canada with postal codes

In [171]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "html.parser")

In [172]:
all_tables=soup.find_all("table")
right_table=soup.find('table', class_='wikitable sortable')

In [173]:
A=[]
B=[]
C=[]


for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

In [174]:

df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C

In [175]:
df = df.replace('\n','', regex=True)

In [176]:
df=df[df.Borough != 'Not assigned']
df=df.reset_index(drop=True)

In [177]:
df.shape

(103, 3)

## Extract location data of neighborhoods  using csv

In [178]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge two tables which have neighborhoods and location data

In [179]:
df_inner = pd.merge(df, df_data_1, on='Postal Code', how='inner')

In [180]:
df_inner.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [181]:
df_inner.shape

(103, 5)

## Extract only details related to Torronto

In [182]:
df_toronto=df_inner[df_inner.Borough.str.contains('Toronto', regex= True, na=False)]
df_toronto=df_toronto.reset_index(drop=True)
df_toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


## use geolocator to get the location of torronto

In [184]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Torronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Torronto are 43.6534817, -79.3839347.


In [185]:
CLIENT_ID = 'NDVY4WVE10S4CQAHEHJVL53ZDGNJ1GG5EFGU2GLJPIDUJ3LV' # your Foursquare ID
CLIENT_SECRET = 'TM5WZTDOCN1ZOIL2FGABQ5DBH2TA0Q4G3HT3C2CJDPCDDYXK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NDVY4WVE10S4CQAHEHJVL53ZDGNJ1GG5EFGU2GLJPIDUJ3LV
CLIENT_SECRET:TM5WZTDOCN1ZOIL2FGABQ5DBH2TA0Q4G3HT3C2CJDPCDDYXK


In the above I have used regex to obtain the relevant data of torronto of difeerent parts like west torronto.Therefore I get sufficient data.

# Plotted neighbourhoods

In [186]:
map_torronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_torronto)  
    
map_torronto

## Extract neighbor hoods around torronto (1km2) using foursquare

In [187]:
neighborhood_latitude = df_toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_toronto.loc[0, 'Neighborhood'] 

In below Analyse all venues near all neighbourhoods

In [188]:
radius = 1000
LIMIT = 300

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=NDVY4WVE10S4CQAHEHJVL53ZDGNJ1GG5EFGU2GLJPIDUJ3LV&client_secret=TM5WZTDOCN1ZOIL2FGABQ5DBH2TA0Q4G3HT3C2CJDPCDDYXK&v=20180605&ll=43.6542599,-79.3606359&radius=1000&limit=300'

In [224]:
results = requests.get(url).json()

In [191]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [192]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Impact Kitchen,Restaurant,43.656369,-79.356980
3,The Distillery Historic District,Historic Site,43.650244,-79.359323
4,Corktown Common,Park,43.655618,-79.356211


In [193]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## Extracting nearby venues

In [194]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URLresults.size()
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [195]:
torronton_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


In [196]:
print(torronton_venues.shape)

(3198, 7)


In [197]:
torronton_venues['Venue Category'].unique()

array(['Bakery', 'Coffee Shop', 'Restaurant', 'Historic Site', 'Park',
       'Distribution Center', 'Farmers Market', 'Breakfast Spot',
       'Chocolate Shop', 'Mediterranean Restaurant', 'Dessert Shop',
       'Spa', 'Performing Arts Venue', 'Liquor Store', 'Tech Startup',
       'Italian Restaurant', 'French Restaurant', 'Café', 'Pub',
       'Gym / Fitness Center', 'Thai Restaurant', 'Pool', 'Shoe Store',
       'Yoga Studio', 'Ice Cream Shop', 'Theater', 'Event Space',
       'Gastropub', 'German Restaurant', 'Furniture / Home Store',
       'Animal Shelter', 'Diner', 'Mexican Restaurant', 'Cosmetics Shop',
       'Art Gallery', 'Sandwich Place', 'Food Truck', 'Gym Pool',
       'Grocery Store', 'Middle Eastern Restaurant', 'Pizza Place',
       'Brewery', 'Dance Studio', 'Karaoke Bar', 'Athletics & Sports',
       'Bar', 'Bank', 'Sushi Restaurant', 'Pet Store',
       'Electronics Store', 'Auto Dealership', 'Beer Store', 'Museum',
       'Pakistani Restaurant', 'Indian Restauran

# Better place for restaurant

### Calculating number of restaurants in each neighborhood

In [198]:
restFreq=torronton_venues[torronton_venues['Venue Category'].str.contains('Restaurant', regex= True, na=False)].groupby('Neighborhood').count().sort_values('Venue', ascending=False)
restFreq

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,37,37,37,37,37,37
"The Danforth West, Riverdale",33,33,33,33,33,33
Christie,33,33,33,33,33,33
"Little Portugal, Trinity",32,32,32,32,32,32
"The Annex, North Midtown, Yorkville",29,29,29,29,29,29
"Garden District, Ryerson",27,27,27,27,27,27
Studio District,27,27,27,27,27,27
"Queen's Park, Ontario Provincial Government",27,27,27,27,27,27
"Brockton, Parkdale Village, Exhibition Place",26,26,26,26,26,26


### Make dataframe with Neighborhood and Venue Category

In [200]:
restFreq=restFreq.reset_index()[['Neighborhood','Venue Category']]

### Divide the dataframe to three sections based on the number of restaurants.

In [201]:
restFreq.columns = ['Neighborhood', 'Restaurant_freq']
concentrated=restFreq[restFreq['Restaurant_freq']>24]
bool_series = restFreq["Restaurant_freq"].between(15, 24, inclusive = True) 
lower=restFreq[restFreq['Restaurant_freq']<15]

### reconstruct table structure to use kmeans algorithem and add different items as features. going to cluster simmilar groups

In [206]:
# one hot encoding
torronton_onehot = pd.get_dummies(torronton_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
torronton_onehot['Neighborhood'] =torronton_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [torronton_onehot.columns[-1]] + list(torronton_onehot.columns[:-1])
torronton_onehot = torronton_onehot[fixed_columns]

torronton_onehot.head()

,Zoo,Accessories Store,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,...,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [207]:
torronton_onehot.shape

(3198, 279)

In [208]:
torronton_grouped = torronton_onehot.groupby('Neighborhood').mean().reset_index()
torronton_grouped

,Neighborhood,Zoo,Accessories Store,Airport,Airport Lounge,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,...,Turkish Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Yoga Studio
0,Berczy Park,0.00,0.00,0.0000,0.0000,0.010000,0.00,0.00,0.00,0.00,...,0.00,0.00,0.010000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.00,0.01,0.0000,0.0000,0.010000,0.00,0.00,0.00,0.00,...,0.00,0.00,0.020000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000
2,"Business reply mail Processing Centre, South C...",0.00,0.00,0.0000,0.0000,0.021277,0.00,0.00,0.00,0.00,...,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0.00,0.0625,0.0625,0.000000,0.00,0.00,0.00,0.00,...,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000
4,Central Bay Street,0.00,0.00,0.0000,0.0000,0.010000,0.00,0.00,0.00,0.00,...,0.00,0.01,0.020000,0.00,0.00,0.000000,0.00,0.000000,0.01,0.020000
5,Christie,0.00,0.00,0.0000,0.0000,0.010000,0.00,0.00,0.00,0.00,...,0.00,0.00,0.020000,0.00,0.00,0.010000,0.00,0.010000,0.00,0.000000
6,Church and Wellesley,0.00,0.00,0.0000,0.0000,0.010000,0.00,0.00,0.00,0.00,...,0.00,0.00,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.01,0.020000
7,"Commerce Court, Victoria Hotel",0.00,0.00,0.0000,0.0000,0.030000,0.00,0.00,0.00,0.00,...,0.00,0.00,0.020000,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000
8,Davisville,0.00,0.00,0.0000,0.0000,0.000000,0.00,0.00,0.00,0.00,...,0.00,0.00,0.010000,0.01,0.00,0.010000,0.00,0.010000,0.00,0.010000
9,Davisville North,0.00,0.00,0.0000,0.0000,0.000000,0.00,0.00,0.00,0.00,...,0.00,0.00,0.010000,0.00,0.00,0.010000,0.00,0.010000,0.00,0.020000


In [209]:
torronton_grouped.shape

(39, 279)

### Extract top venues in torronton

In [210]:
num_top_venues = 5

for hood in torronton_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = torronton_grouped[torronton_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                 venue  freq
0          Coffee Shop  0.12
1                 Café  0.07
2  Japanese Restaurant  0.04
3           Restaurant  0.04
4                Hotel  0.04


----Brockton, Parkdale Village, Exhibition Place----
         venue  freq
0         Café  0.07
1  Coffee Shop  0.06
2          Bar  0.05
3   Restaurant  0.05
4       Bakery  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                venue  freq
0                Park  0.09
1         Pizza Place  0.06
2             Brewery  0.06
3         Coffee Shop  0.06
4  Italian Restaurant  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0              Café  0.12
1       Coffee Shop  0.12
2   Harbor / Marina  0.12
3  Sushi Restaurant  0.06
4    Scenic Lookout  0.06


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.08
1 

In [155]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Each neighborhood with common popular venues

In [211]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = torronton_grouped['Neighborhood']

for ind in np.arange(torronton_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(torronton_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Japanese Restaurant,Restaurant,Hotel,Park,Beer Bar,Bakery,New American Restaurant,Cocktail Bar
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Bar,Restaurant,Bakery,Furniture / Home Store,Gift Shop,Tibetan Restaurant,Art Gallery,Park
2,"Business reply mail Processing Centre, South C...",Park,Brewery,Pizza Place,Coffee Shop,Fast Food Restaurant,Sushi Restaurant,Italian Restaurant,Pub,Restaurant,Comic Shop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Harbor / Marina,Café,Coffee Shop,Garden,Track,Scenic Lookout,Sushi Restaurant,Sculpture Garden,Dance Studio,Park
4,Central Bay Street,Coffee Shop,Café,Clothing Store,Park,Art Gallery,Japanese Restaurant,Furniture / Home Store,Hotel,Sushi Restaurant,Italian Restaurant


### Applying kmeans algorithem to above dataset

In [212]:
# set number of clusters
kclusters = 5

torronton_grouped_clustering = torronton_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(torronton_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 3, 4, 0, 0, 0, 2, 0, 0], dtype=int32)

In [213]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

torronton_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
torronton_merged = torronton_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

torronton_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Theater,Diner,Café,Restaurant,Pub,Italian Restaurant,Park,Breakfast Spot,Bakery
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Burger Joint,Park,Japanese Restaurant,Italian Restaurant,Ramen Restaurant,Bookstore,Clothing Store,Pizza Place
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Coffee Shop,Gastropub,Japanese Restaurant,Theater,Restaurant,Hotel,Italian Restaurant,Burrito Place,Café,Ramen Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Restaurant,Gastropub,Theater,Seafood Restaurant,Italian Restaurant,Cocktail Bar,Cosmetics Shop,Concert Hall
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Pub,Pizza Place,Japanese Restaurant,Park,Breakfast Spot,Beach,Bakery,Bar,Burger Joint


### Plot each cluster with different colors

In [214]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(torronton_merged['Latitude'], torronton_merged['Longitude'], torronton_merged['Neighborhood'], torronton_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [215]:
torronton_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Theater,Diner,Café,Restaurant,Pub,Italian Restaurant,Park,Breakfast Spot,Bakery
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Sushi Restaurant,Burger Joint,Park,Japanese Restaurant,Italian Restaurant,Ramen Restaurant,Bookstore,Clothing Store,Pizza Place
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,2,Coffee Shop,Gastropub,Japanese Restaurant,Theater,Restaurant,Hotel,Italian Restaurant,Burrito Place,Café,Ramen Restaurant
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,Coffee Shop,Café,Restaurant,Gastropub,Theater,Seafood Restaurant,Italian Restaurant,Cocktail Bar,Cosmetics Shop,Concert Hall
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Pub,Pizza Place,Japanese Restaurant,Park,Breakfast Spot,Beach,Bakery,Bar,Burger Joint


### Calculating number of restaurants from the earlier group as fraction inside clusters

In [216]:
def return_most_common_cluster(torrontonCluster, restaurantCluster):
    list1=[]
    for x in range(5): 
        cluster1=torrontonCluster[torrontonCluster['Cluster Labels'] == x]
        mergedRestaurant1 = pd.merge(cluster1,restaurantCluster, on=['Neighborhood'], how='inner')
       
        list1.append(mergedRestaurant1.count()['Restaurant_freq']/(restaurantCluster.count()['Restaurant_freq']))
    return list1

### Calculate concentrated group as fraction in all 5 clusters

In [217]:
return_most_common_cluster(torronton_merged,concentrated)

[0.8333333333333334, 0.0, 0.16666666666666666, 0.0, 0.0]

### Calculate lower group as fraction in all 5 clusters(less than 12 restaurants)

In [218]:
return_most_common_cluster(torronton_merged,lower)

[0.2222222222222222,
 0.1111111111111111,
 0.0,
 0.5555555555555556,
 0.1111111111111111]

### Calculate medium group as fraction in all 5 clusters(12<restaurants<24)

In [219]:
return_most_common_cluster(torronton_merged,restFreq[bool_series])

[0.4444444444444444, 0.0, 0.4444444444444444, 0.1111111111111111, 0.0]

When consider above values cluster 0 is better place for restaurants .we could observe that there are neigbourhoods which are in the same cluster 0 but they have very less number of restaurants. Therefore surely better to start new restaurant on those neighbourhoods. Lets looks into cluster 0

In [220]:
cluster1_selected=torronton_merged[torronton_merged['Cluster Labels'] == 0]

In [221]:
pd.merge(cluster1_selected,lower, on=['Neighborhood'], how='inner')


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Restaurant_freq
0,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678,0,Coffee Shop,Skating Rink,Italian Restaurant,Diner,Mexican Restaurant,Café,Restaurant,Clothing Store,Park,Sporting Goods Shop,11
1,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675,0,Café,Park,Japanese Restaurant,Gastropub,Restaurant,Diner,Filipino Restaurant,Steakhouse,Taiwanese Restaurant,Deli / Bodega,11
